# Introduction

In this notebook, I will play with FuzzyMarketState python class, testing all its features.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import requires packages
# Append relative path to FuzzyMarketState.py
import sys
sys.path.append('..')

from FuzzyMarketState import FuzzyMarketState

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import plotly.tools as tls
plotly.offline.init_notebook_mode(connected=True)

import logging
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)

import random

print('Packages loaded!!')

Packages loaded!!
DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [3]:
# Create object
fms = FuzzyMarketState(logging.WARN)

In [4]:
# load dataframe
df = fms.loadCSV('../csv_data/EURUSD_H4.csv', sep=';')

## Zigzag and Bollinger indicators

In [7]:
# build zigzag indicator
df = fms.buildZigzag(df, minbars=12, bb_period=20, bb_dev=2.0, nan_value=0.0)


In [8]:
# load dataframe with zigzag into fms and plot the zigzag trace
fms.loadDataframe(df)
zz_traces = fms.plotZigzag('black')
py.iplot(zz_traces, filename='simple_ohlc')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


In [10]:
# plots Bollinger indicators associated with zigzag
fig, bb_traces = fms.plotBollinger(['black', 'blue', 'red'])
py.iplot(fig, filename='simple_ohlc')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Oscillators: MACD and RSI

In [11]:
result = fms.buildOscillators(fms.getDataframe(), macd_applied='CLOSE', macd_fast=12, macd_slow=26, macd_sig=9, rsi_applied='CLOSE', rsi_period=14)

In [12]:
fig,osc_traces = fms.plotOscillators(color=['blue','red','green'])
py.iplot(fig, filename='simple_ohlc')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Moving Averages Fast-Medium-Slow

In [13]:
# build moving averages
result = fms.build3MovingAverages(fms.getDataframe(), 
                                  ma_fast_applied='CLOSE', ma_fast_period=50, ma_fast_type='SMA', 
                                  ma_mid_applied='CLOSE', ma_mid_period=100, ma_mid_type='SMA', 
                                  ma_slow_applied='CLOSE', ma_slow_period=200, ma_slow_type='SMA',
                                  trend_filters={'price-slow': 0.5, 'price-mid': 0.3, 'price-fast': 0.2})

In [15]:
# plot moving averages
fig, ma_traces, ma_shapes = fms.plotMovingAverages(color=['blue', 'red', 'green'])
#-------------
#def zoom(layout, xrange):
#    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
#    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
#fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Fibonacci levels

In [16]:
# build common fibo levels (both retracements and extensions)
fms.buildCommonFiboLevels(fms.getDataframe(), nan_value=0.0)

In [17]:
# plot fibo levels for a given sample
length =  fms.getDataframe().shape[0]
x = random.randint(int(length/2), length)
fibo_traces, fibo_annotations, fibo_shapes = fms.plotFiboLevels(at=x, width=100, color='black')
fig = go.Figure(data=[fibo_traces])
fig['layout'].update(height=600, title='FiboLevels at {}'.format(x))
fig['layout'].update(annotations=fibo_annotations)
fig['layout'].update(shapes=fibo_shapes)
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Supports & Resistances

In [18]:
# build supports and resistances levels from last trend zigzags
_supports = fms.buildSupports(fms.getDataframe())

(6182, 6138)

In [19]:
# plot supports levels for samples x1 and x2
s = _supports
s = s[s != 0.0]
x1 = s.index.values[-1]
x2 = s.index.values[-2]

trace_ohlc,s1_shape=fms.plotHorizontalLine(x1+1, x1, x1+100, s.iloc[-1], color='brown', width=2, dash='dashdot')
_,s2_shape=fms.plotHorizontalLine(x2+1, x2, x2+100, s.iloc[-2], color='violet', width=2, dash='dashdot')
fig = go.Figure(data=[trace_ohlc])
fig['layout'].update(height=600, title='Supports at {},{}'.format(x1+1,x2+1))
fig['layout'].update(shapes=[s1_shape, s2_shape])
#-------------
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


In [20]:
# build supports and resistances levels from last trend zigzags
_resist = fms.buildResistances(fms.getDataframe(), nan_value=0.0)


In [21]:
# plot supports levels for samples x1 and x2
r = _resist
r = r[r != 0.0]
x1 = r.index.values[-1]
x2 = r.index.values[-2]

trace_ohlc,r1_shape=fms.plotHorizontalLine(at=x1+1, x0=x1, x1=x1+100, value=r.iloc[-1], color='brown', width=2, dash='dashdot')
_,r2_shape=fms.plotHorizontalLine(x2+1, x2, x2+100, r.iloc[-2], color='violet', width=2, dash='dashdot')
fig = go.Figure(data=[trace_ohlc])
fig['layout'].update(height=600, title='Resistances at {},{}'.format(x1+1,x2+1))
fig['layout'].update(shapes=[r1_shape, r2_shape])
#-------------
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Channels

In [22]:
# build channel lines
_channel = fms.buildChannel(fms.getDataframe(), nan_value='')

In [23]:
# plot channel lines for sample _ux or _bx
_upperline = _channel['channel_upper_limit']
_bottomline = _channel['channel_lower_limit']
_upperline = _upperline[_upperline != '']
_bottomline = _bottomline[_bottomline != '']
_ux = _upperline.index.values[-1]
_bx = _bottomline.index.values[-1]
_ulast = _upperline.iloc[-1]
_blast = _bottomline.iloc[-1]

x = _ux+1
trace_ohlc, ch_shapes = fms.plotChannel(x, extended=100, color='black', width=1, dash='dashdot')
fig = go.Figure(data=[trace_ohlc])
fig['layout'].update(height=600, title='Channel at {}'.format(x))
fig['layout'].update(shapes=ch_shapes)
#-------------
def zoom(layout, xrange):
    in_view = df.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.High.min() - 10, in_view.High.max() + 10]
fig.layout.on_change(zoom, 'xaxis.range')
py.iplot(fig, filename='simple_ohlc')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Trends

In [24]:
# build trends with different filters
filters = {'sma-trend':0.75, 'zigzag-trend':0.15, 'fibo-trend':0.1}
_trends = fms.buildTrends(fms.getDataframe(), filters = filters, nan_value=0.0)

In [26]:
# plot trends
fig, trace_ohlc, trend_shapes = fms.plotTrends(nan_value=0.0)
py.iplot(fig, filename='simple_ohlc')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Divergences

In [94]:
# build divergences
fms.setLoggingLevel(logging.DEBUG)
_bullish_div, _bearish_div = fms.buildDivergences(fms.getDataframe(), nan_value=0.0)

DEBUG:FuzzyMarketState:Logging level changed to 10
DEBUG:FuzzyMarketState:row [0]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [1]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [2]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [3]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [4]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [5]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [6]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [7]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [8]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [9]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [10]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [11]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [12]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [13]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [14]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [15]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [16]: error-zzpoints-count
DEBUG:FuzzyMarketState:row [17]: error-zz

DEBUG:FuzzyMarketState:row [145]: last is MIN p0=145, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6-isNaN 
DEBUG:FuzzyMarketState:row [146]: last is MIN p0=146, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6-isNaN 
DEBUG:FuzzyMarketState:row [147]: last is MIN p0=147, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6-isNaN 
DEBUG:FuzzyMarketState:row [148]: last is MIN p0=148, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6-isNaN 
DEBUG:FuzzyMarketState:row [149]: last is MIN p0=149, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6-isNaN 
DEBUG:FuzzyMarketState:row [150]: last is MIN p0=150, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6-isNaN 
DEBUG:FuzzyMarketState:row [151]: last is MIN p0=151, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6-isNaN 
DEBUG:FuzzyMarketState:row [152]: last is MIN p0=152, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6-isNaN 
DEBUG:FuzzyMarketState:row [153]: last is MIN p0=153, p1=139, p2=112, p3=76, p4=58, p5=44, p6=0 error-P6

DEBUG:FuzzyMarketState:row [225]: last is MAX p0=225, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [226]: last is MAX p0=226, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [227]: last is MAX p0=227, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [228]: last is MAX p0=228, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [229]: last is MAX p0=229, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [230]: last is MAX p0=230, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [231]: last is MAX p0=231, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [232]: last is MAX p0=232, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [233]: last is MAX p0=233, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:FuzzyMarketState:row [234]: last is MAX p0=234, p1=172, p2=139, p3=112, p4=76, p5=58, p6=44 
DEBUG:Fuzz

DEBUG:FuzzyMarketState:row [304]: last is MIN p0=304, p1=285, p2=172, p3=139, p4=112, p5=76, p6=58 reg-bull rng1=48 rng2=21
DEBUG:FuzzyMarketState:row [305]: last is MIN p0=305, p1=285, p2=172, p3=139, p4=112, p5=76, p6=58 reg-bull rng1=48 rng2=21
DEBUG:FuzzyMarketState:row [306]: last is MIN p0=306, p1=285, p2=172, p3=139, p4=112, p5=76, p6=58 reg-bull rng1=48 rng2=21
DEBUG:FuzzyMarketState:row [307]: last is MIN p0=307, p1=285, p2=172, p3=139, p4=112, p5=76, p6=58 reg-bull rng1=48 rng2=21
DEBUG:FuzzyMarketState:row [308]: last is MIN p0=308, p1=285, p2=172, p3=139, p4=112, p5=76, p6=58 reg-bull rng1=48 rng2=21
DEBUG:FuzzyMarketState:row [309]: last is MIN p0=309, p1=285, p2=172, p3=139, p4=112, p5=76, p6=58 reg-bull rng1=48 rng2=21
DEBUG:FuzzyMarketState:row [310]: last is MIN p0=310, p1=285, p2=172, p3=139, p4=112, p5=76, p6=58 reg-bull rng1=48 rng2=21
DEBUG:FuzzyMarketState:row [311]: last is MIN p0=311, p1=285, p2=172, p3=139, p4=112, p5=76, p6=58 reg-bull rng1=48 rng2=21
DEBUG:Fu

DEBUG:FuzzyMarketState:row [355]: last is MAX p0=355, p1=334, p2=285, p3=172, p4=139, p5=112, p6=76 hid-bear rng1=54 rng2=20hid-bear-div on macd ifrom 120 to 324hid-bear-div on rsi ifrom 119 to 336reg-bull rng1=48 rng2=21
DEBUG:FuzzyMarketState:row [356]: last is MIN p0=356, p1=355, p2=334, p3=285, p4=172, p5=139, p6=112 hid-bear rng1=54 rng2=20hid-bear-div on macd ifrom 120 to 324hid-bear-div on rsi ifrom 119 to 336
DEBUG:FuzzyMarketState:row [357]: last is MIN p0=357, p1=355, p2=334, p3=285, p4=172, p5=139, p6=112 hid-bear rng1=54 rng2=20hid-bear-div on macd ifrom 120 to 324hid-bear-div on rsi ifrom 119 to 336
DEBUG:FuzzyMarketState:row [358]: last is MIN p0=358, p1=355, p2=334, p3=285, p4=172, p5=139, p6=112 hid-bear rng1=54 rng2=20hid-bear-div on macd ifrom 120 to 324hid-bear-div on rsi ifrom 119 to 336
DEBUG:FuzzyMarketState:row [359]: last is MIN p0=359, p1=355, p2=334, p3=285, p4=172, p5=139, p6=112 hid-bear rng1=54 rng2=20hid-bear-div on macd ifrom 120 to 324hid-bear-div on rsi

...

DEBUG:FuzzyMarketState:row [6562]: last is MAX p0=6562, p1=6534, p2=6480, p3=6448, p4=6396, p5=6329, p6=6302 reg-bull rng1=28 rng2=31
DEBUG:FuzzyMarketState:row [6563]: last is MAX p0=6563, p1=6534, p2=6480, p3=6448, p4=6396, p5=6329, p6=6302 reg-bull rng1=28 rng2=31
DEBUG:FuzzyMarketState:row [6564]: last is MAX p0=6564, p1=6534, p2=6480, p3=6448, p4=6396, p5=6329, p6=6302 reg-bull rng1=28 rng2=31
DEBUG:FuzzyMarketState:row [6565]: last is MAX p0=6565, p1=6534, p2=6480, p3=6448, p4=6396, p5=6329, p6=6302 reg-bull rng1=28 rng2=31
DEBUG:FuzzyMarketState:row [6566]: last is MAX p0=6566, p1=6534, p2=6480, p3=6448, p4=6396, p5=6329, p6=6302 reg-bull rng1=28 rng2=31
DEBUG:FuzzyMarketState:row [6567]: last is MAX p0=6567, p1=6534, p2=6480, p3=6448, p4=6396, p5=6329, p6=6302 reg-bull rng1=28 rng2=31
DEBUG:FuzzyMarketState:row [6568]: last is MAX p0=6568, p1=6534, p2=6480, p3=6448, p4=6396, p5=6329, p6=6302 reg-bull rng1=28 rng2=31
DEBUG:FuzzyMarketState:row [6569]: last is MAX p0=6569, p1=653

DEBUG:FuzzyMarketState:row [6624]: last is MIN p0=6624, p1=6587, p2=6534, p3=6480, p4=6448, p5=6396, p6=6329 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6625]: last is MIN p0=6625, p1=6587, p2=6534, p3=6480, p4=6448, p5=6396, p6=6329 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6626]: last is MIN p0=6626, p1=6587, p2=6534, p3=6480, p4=6448, p5=6396, p6=6329 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6627]: last is MIN p0=6627, p1=6587, p2=6534, p3=6480, p4=6448, p5=6396, p6=6329 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6628]: last is MIN p0=6628, p1=6587, p2=6534, p3=6480, p4=6448, p5=6396, p6=6329 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6629]: last is MIN p0=6629, p1=6587, p2=6534, p3=6480, p4=6448, p5=6396, p6=6329 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6630]: last is MIN p0=6630, p1=6587, p2=6534, p3=6480, p4=6448, p5=6396, p6=6329 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6631]: last is MIN p0=6631, p1=658

DEBUG:FuzzyMarketState:row [6686]: last is MAX p0=6686, p1=6635, p2=6587, p3=6534, p4=6480, p5=6448, p6=6396 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6687]: last is MAX p0=6687, p1=6635, p2=6587, p3=6534, p4=6480, p5=6448, p6=6396 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6688]: last is MAX p0=6688, p1=6635, p2=6587, p3=6534, p4=6480, p5=6448, p6=6396 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6689]: last is MAX p0=6689, p1=6635, p2=6587, p3=6534, p4=6480, p5=6448, p6=6396 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6690]: last is MAX p0=6690, p1=6635, p2=6587, p3=6534, p4=6480, p5=6448, p6=6396 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6691]: last is MAX p0=6691, p1=6635, p2=6587, p3=6534, p4=6480, p5=6448, p6=6396 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6692]: last is MAX p0=6692, p1=6635, p2=6587, p3=6534, p4=6480, p5=6448, p6=6396 hid-bull rng1=35 rng2=28
DEBUG:FuzzyMarketState:row [6693]: last is MAX p0=6693, p1=663

DEBUG:FuzzyMarketState:row [6739]: last is MIN p0=6739, p1=6724, p2=6714, p3=6694, p4=6635, p5=6587, p6=6534 hid-bear rng1=26 rng2=33hid-bear-div on rsi ifrom 6634 to 6713
DEBUG:FuzzyMarketState:row [6740]: last is MIN p0=6740, p1=6724, p2=6714, p3=6694, p4=6635, p5=6587, p6=6534 hid-bear rng1=26 rng2=33hid-bear-div on rsi ifrom 6634 to 6713
DEBUG:FuzzyMarketState:row [6741]: last is MIN p0=6741, p1=6724, p2=6714, p3=6694, p4=6635, p5=6587, p6=6534 hid-bear rng1=26 rng2=33hid-bear-div on rsi ifrom 6634 to 6713
DEBUG:FuzzyMarketState:row [6742]: last is MIN p0=6742, p1=6724, p2=6714, p3=6694, p4=6635, p5=6587, p6=6534 hid-bear rng1=26 rng2=33hid-bear-div on rsi ifrom 6634 to 6713
DEBUG:FuzzyMarketState:row [6743]: last is MIN p0=6743, p1=6724, p2=6714, p3=6694, p4=6635, p5=6587, p6=6534 hid-bear rng1=26 rng2=33hid-bear-div on rsi ifrom 6634 to 6713
DEBUG:FuzzyMarketState:row [6744]: last is MIN p0=6744, p1=6724, p2=6714, p3=6694, p4=6635, p5=6587, p6=6534 hid-bear rng1=26 rng2=33hid-bea

DEBUG:FuzzyMarketState:row [6801]: last is MIN p0=6801, p1=6795, p2=6761, p3=6724, p4=6714, p5=6694, p6=6635 
DEBUG:FuzzyMarketState:row [6802]: last is MIN p0=6802, p1=6795, p2=6761, p3=6724, p4=6714, p5=6694, p6=6635 
DEBUG:FuzzyMarketState:row [6803]: last is MIN p0=6803, p1=6795, p2=6761, p3=6724, p4=6714, p5=6694, p6=6635 
DEBUG:FuzzyMarketState:row [6804]: last is MIN p0=6804, p1=6795, p2=6761, p3=6724, p4=6714, p5=6694, p6=6635 
DEBUG:FuzzyMarketState:row [6805]: last is MIN p0=6805, p1=6795, p2=6761, p3=6724, p4=6714, p5=6694, p6=6635 


In [98]:
# plot divergences
fms.setLoggingLevel(logging.WARN)
fig, traces, div_shapes = fms.plotDivergences(color='blue', nan_value = 0.0)
py.iplot(fig, filename='simple_ohlc')

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): plot.ly:443
DEBUG:urllib3.connectionpool:https://plot.ly:443 "POST /clientresp HTTP/1.1" 200 None


### Indicator based API

In [9]:
# load dataframe
df = fms.loadCSV('../csv_data/EURUSD_H4.csv', sep=';')
fms.loadDataframe(df)
_df = fms.buildIndicators()
fms.plotIndicators()

In [ ]:
fig = go.Figure(data=[fms.trace_ohlc, fms.trace_zigzag])
fig['layout'].update(height=600, title='Bollinger indicators')
fig['layout'].update(shapes=fms.trend_shapes)
py.iplot(fig, filename='simple_ohlc')


### New bar update

In [26]:
# load dataframe
df = fms.loadCSV('../csv_data/EURUSD_H4.csv', sep=';')
_df1 = df[:-1].copy()
fms.loadDataframe(_df1)
_df2 = fms.buildIndicators()
_df2.tail()

,TIME,OPEN,HIGH,LOW,CLOSE,TICKVOL,VOL,SPREAD,BOLLINGER_HI,BOLLINGER_MA,...,DIV_REG_BULL_RSI,DIV_REG_BULL_RSI_FROM,DIV_DOUB_HID_BEAR_RSI,DIV_DOUB_HID_BEAR_RSI_FROM,DIV_HID_BEAR_RSI,DIV_HID_BEAR_RSI_FROM,DIV_DOUB_HID_BULL_RSI,DIV_DOUB_HID_BULL_RSI_FROM,DIV_HID_BULL_RSI,DIV_HID_BULL_RSI_FROM
6520,2019-05-22 04:00:00,1.11608,1.11656,1.11533,1.11553,3295,0,0,1.116355,1.115805,...,0,0,0,0,0,0,0,0,0,0
6521,2019-05-22 08:00:00,1.11553,1.11638,1.11486,1.11594,10759,0,0,1.116145,1.115735,...,0,0,0,0,0,0,0,0,0,0
6522,2019-05-22 12:00:00,1.11594,1.11804,1.11579,1.11669,11949,0,0,1.117065,1.116315,...,0,0,0,0,0,0,0,0,0,0
6523,2019-05-22 16:00:00,1.11669,1.11689,1.11509,1.11569,15134,0,0,1.117190,1.116190,...,0,0,0,0,0,0,0,0,0,0
6524,2019-05-22 20:00:00,1.11569,1.11636,1.11495,1.11495,5914,0,0,1.116060,1.115320,...,0,0,0,0,0,0,0,0,0,0


In [27]:
row = df.iloc[-1].copy()
_df3 = fms.updateIndicators(row)
_df3.tail()

,TIME,OPEN,HIGH,LOW,CLOSE,TICKVOL,VOL,SPREAD,BOLLINGER_HI,BOLLINGER_MA,...,DIV_REG_BULL_RSI,DIV_REG_BULL_RSI_FROM,DIV_DOUB_HID_BEAR_RSI,DIV_DOUB_HID_BEAR_RSI_FROM,DIV_HID_BEAR_RSI,DIV_HID_BEAR_RSI_FROM,DIV_DOUB_HID_BULL_RSI,DIV_DOUB_HID_BULL_RSI_FROM,DIV_HID_BULL_RSI,DIV_HID_BULL_RSI_FROM
6521,2019-05-22 08:00:00,1.11553,1.11638,1.11486,1.11594,10759,0,0,1.116145,1.115735,...,0,0,0,0,0,0,0,0,0,0
6522,2019-05-22 12:00:00,1.11594,1.11804,1.11579,1.11669,11949,0,0,1.117065,1.116315,...,0,0,0,0,0,0,0,0,0,0
6523,2019-05-22 16:00:00,1.11669,1.11689,1.11509,1.11569,15134,0,0,1.117190,1.116190,...,0,0,0,0,0,0,0,0,0,0
6524,2019-05-22 20:00:00,1.11569,1.11636,1.11495,1.11495,5914,0,0,1.116060,1.115320,...,0,0,0,0,0,0,0,0,0,0
6525,2019-05-23 00:00:00,1.11495,1.11571,1.11495,1.11525,3558,0,0,1.115400,1.115100,...,0,0,0,0,0,0,0,0,0,0
